In [ ]:
import pathlib
import pandas as pd

datadir = pathlib.Path.cwd().parent / "data"

pd_hotel       = pd.read_parquet(datadir / "hotel.parquet")
pd_customer    = pd.read_parquet(datadir / "customer.parquet")
pd_reservation = pd.read_parquet(datadir / "reservation.parquet")

# 5章 抽出
## 5-1 列名指定による列の選択


### Q: 集計対象の列のみに絞り込み


#### Not Awesome 1


In [ ]:
# 不要な列を指定して削除
pd_reservation.drop(columns=[
    "reserved_at",
    "length_of_stay",
    "total_price",
    "people_num",
    "status",
    "canceled_at",
])

#### Not Awesome 2


In [ ]:
# インデックスで選択
pd_reservation.iloc[:, [0,1,2,4,5]]

#### Awesome


In [ ]:
# 列名を指定して列を選択
pd_reservation[[
    "reservation_id",
    "hotel_id",
    "customer_id",
    "checkin_date",
    "checkout_date",
]]

In [ ]:
# 列名で指定して列を選択
pd_reservation.loc[:, [
    "reservation_id",
    "hotel_id",
    "customer_id",
    "checkin_date",
    "checkout_date",
]]

## 5-2 条件指定による列の抽出


### Q 列名が`tag`から始まる列の抽出
#### Not Awesome


In [ ]:
# 列名を列挙して抽出
cols = [
    "tag_001",
    "tag_002",
    "tag_003",
    # （中略）
    "tag_028",
    "tag_029",
    "tag_030",
]
pd_hotel[cols]


#### Awesome


In [ ]:
# 列の開始／終了の部分一致ルールを指定して選択
pd_hotel.loc[:, lambda df: df.columns.str.startswith("tag_")]

### Q 欠損のある列の抽出


#### Not Awesome


In [ ]:
# （1）列の情報（非欠損値の数）を確認
pd_customer.info()

In [ ]:

# （2）欠損値を含む列を指定して選択
cols = [
    "sex",
    "address_town",
]
pd_customer[cols]

#### Awesome


In [ ]:
# 欠損値を含む列を条件判定して抽出
pd_customer.loc[:, lambda df: df.isnull().any()]

### Q 数値型の列の抽出


#### Not Awesome


In [ ]:
# データ型を確認してから列名を列挙して抽出
pd_reservation.info()

In [ ]:
cols = [
    "reservation_id",
    "hotel_id",
    "customer_id",
    "length_of_stay",
    "people_num",
    "total_price",
]
pd_reservation[cols]

#### Awesome


In [ ]:
# select_dtypesで数値型の列をすべて選択
pd_reservation.select_dtypes("number")

In [ ]:
# select_dtypesでobject型の列をすべて選択
pd_reservation.select_dtypes("object")

In [ ]:
# select_dtypesでdatetime型の列をすべて選択
pd_reservation.select_dtypes(["datetime", "datetimetz"])

## 5-3 条件指定による行の抽出


### Q: 宿泊人数が2〜4人の予約履歴のみ抽出


#### Not Awesome


In [ ]:
# -- people_numが2以上、かつ4以下の行を抽出
pd_reservation[(pd_reservation.people_num >= 2) & (pd_reservation.people_num <= 4)]

#### Awesome 1


In [ ]:
# people_numが2から4の範囲の行を抽出
pd_reservation.query("2 <= people_num <= 4")

#### Awesome 2


In [ ]:
# people_numが2から4の範囲の行を抽出
pd_reservation.loc[lambda df: df.people_num.between(2, 4)]

## 5-4 ランダムサンプリング


### Q: 予約履歴をランダムサンプリング

#### Not Awesome


In [ ]:
# 先頭の20000行を抽出
pd_reservation.head(20000)

#### Awesome


In [ ]:
# 20000件をランダムサンプリング
pd_reservation.sample(20000)

## 5-5 不均衡データの調整


### Q: 未キャンセルデータをキャンセル済データと同数になるようにアンダーサンプリング


#### Awesome 1


In [ ]:
# （1）未キャンセルデータを抽出
majority = pd_reservation.query("status == 'reserved'")

# （2）キャンセル済データを抽出
minority = pd_reservation.query("status == 'canceled'")

# （3）キャンセル済データとランダムサンプリングした未キャンセルデータを結合
pd.concat([
    minority,
    majority.sample(len(minority))
])

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# （1）学習データの特徴量と目的変数を分割
feature_cols = ["length_of_stay", "people_num", "total_price"]
target_col = "status"
X = pd_reservation[feature_cols]
y = pd_reservation[target_col]

# （2）アンダーサンプリングを行うクラスのオブジェクトを作成
sampler = RandomUnderSampler()

# （3）特徴量と目的変数を関数に渡し、アンダーサンプリングを実行
X_sampled, y_sampled = sampler.fit_resample(X, y)

## [コラム] アンダーサンプリング + バギング


In [ ]:
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# （1）学習データの特徴量と目的変数を分割
feature_cols = ["length_of_stay", "people_num", "total_price"]
target_col = "status"
X = pd_reservation[feature_cols]
y = pd_reservation[target_col]

# （2）予測モデル、およびアンダーサンプリング+バギングを行うクラスのオブジェクトを作成
classifer = DecisionTreeClassifier()
model = BalancedBaggingClassifier(estimator=classifer)
# （3）特徴量と目的変数を関数に渡し、学習を実行
model.fit(X, y)